In [31]:
import git
from pathlib import Path
import os
import numpy as np
import tqdm
import scipy.ndimage as ndi
import pandas as pd
from skimage.filters import gabor_kernel
from scipy.signal import fftconvolve

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from learned import *

np.random.seed(0)

In [32]:
DATASET = "agriVision"
RAW_DATA_SUFFIX = "batch0-agriVision-RGB-cleaned-jitter 2"
FINAL_DATA_NAME = "agriVision-full"
PARAM_CSV = "gabor_new.csv"
CONSTANT_SAMPLE_SIZE = int(1e5)

num_images=1000
jitter=False 
normalize=False 

#data_dir = os.path.join(ROOT_DIR, 'raw-data', DATASET, RAW_DATA_SUFFIX)
data_dir = os.path.join(ROOT_DIR, 'raw-data', RAW_DATA_SUFFIX)

In [33]:
def _generate_gabor_kernel_skimage_cached(frequency: float,
                                          wave_number: int,
                                          theta: float,
                                          aspect_ratio: float,
                                          dtype_str: str) -> np.ndarray:
    if frequency <= 0:
        raise ValueError("frequency must be positive (cycles/pixel).")
    
    # choose n_stds explicitly (same value you pass to gabor_kernel; default is 3)
    n_stds = 3
    sigma_x = wave_number / (2.0 * n_stds * frequency)   # NOT 4*frequency
    sigma_y = aspect_ratio * sigma_x
    k_complex = gabor_kernel(frequency=frequency, theta=theta, sigma_x=sigma_x, sigma_y=sigma_y, n_stds=n_stds)

    k = np.real(k_complex).astype(np.dtype(dtype_str), copy=False)
    k -= k.mean()
    nrm = np.linalg.norm(k.ravel())
    if nrm > 1e-12:
        k = k / nrm
    return k

def generate_gabor_kernel_skimage(frequency: float,
                                  wave_number: int,
                                  theta: float = 0.0,
                                  aspect_ratio: float = 1.0,
                                  dtype=np.float64) -> np.ndarray:
    """Public wrapper returning a copy to avoid accidental mutation of cached data."""
    arr = _generate_gabor_kernel_skimage_cached(
        float(frequency), int(wave_number), float(theta),
        float(aspect_ratio), np.dtype(dtype).name
    )
    if arr is not None:
        return arr.copy()
    return None

In [34]:
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))
# param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", "gabor_test.csv"))

orientations = [0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6]

filters = []
for i in range(len(param_df)):
    fil_orients = []

    for j in range(len(orientations)):
        fil = generate_gabor_kernel_skimage(param_df["frequency"][i], param_df["wave_number"][i], orientations[j], param_df["aspect_ratio"][i])
        if fil is not None:
            fil_orients.append(fil)
    
    filters.append(fil_orients)

print(filters[0][0].shape)
print(len(filters), len(filters[0]))

(25, 47)
42 6


In [35]:
for i in range(len(filters)):
    for j in range(len(filters[i])):
        print(f"index: {i}, params: {tuple(param_df.loc[i])[1:]}, orientation: {orientations[j]:.3f}, size: {filters[i][j].shape}")

index: 0, params: (2.0, 0.5, 0.044), orientation: 0.000, size: (25, 47)
index: 0, params: (2.0, 0.5, 0.044), orientation: 0.524, size: (25, 41)
index: 0, params: (2.0, 0.5, 0.044), orientation: 1.047, size: (41, 25)
index: 0, params: (2.0, 0.5, 0.044), orientation: 1.571, size: (47, 25)
index: 0, params: (2.0, 0.5, 0.044), orientation: 2.094, size: (41, 25)
index: 0, params: (2.0, 0.5, 0.044), orientation: 2.618, size: (25, 41)
index: 1, params: (2.0, 0.5, 0.065), orientation: 0.000, size: (17, 33)
index: 1, params: (2.0, 0.5, 0.065), orientation: 0.524, size: (17, 29)
index: 1, params: (2.0, 0.5, 0.065), orientation: 1.047, size: (29, 17)
index: 1, params: (2.0, 0.5, 0.065), orientation: 1.571, size: (33, 17)
index: 1, params: (2.0, 0.5, 0.065), orientation: 2.094, size: (29, 17)
index: 1, params: (2.0, 0.5, 0.065), orientation: 2.618, size: (17, 29)
index: 2, params: (2.0, 0.5, 0.096), orientation: 0.000, size: (13, 23)
index: 2, params: (2.0, 0.5, 0.096), orientation: 0.524, size: (

In [36]:
images = load_images_from_directory(data_dir, n=num_images, jitter=jitter, normalize=normalize)
print("num_images, H, W, channels:", images.shape, "\nnum_filters", len(filters))

Loading images:   0%|          | 0/1000 [00:00<?, ?it/s]

num_images, H, W, channels: (1000, 512, 512, 3) 
num_filters 42


In [ ]:
def apply_filter(image, kernel):
    """
    Convolve each channel of an image with a kernel, using minimal extra memory.
    """
    # Ensure contiguous float32
    img = np.asarray(image, dtype=np.float32, order="C")
    ker = np.asarray(kernel, dtype=np.float32, order="C")
    out = np.empty_like(img)

    # Handle grayscale or RGB seamlessly
    if img.ndim == 2:  # grayscale
        out[...] = fftconvolve(img, ker, mode="same").astype(np.float32, copy=False)
    
    else:
        # Color image
        C = img.shape[2]
        for ch in range(C):
            out[..., ch] = fftconvolve(img[..., ch], ker, mode="same").astype(np.float32, copy=False)

    return out

In [ ]:
filter_data_map = dict()
images_per_orient = 166 # ~= 1000 / 6

for i in tqdm(range(len(filters))):
    transformed = []
    for j in range(len(filters[i])):   
        curr_transformed = []    
        fil = filters[i][j]

        indices = np.random.choice(len(images), size=images_per_orient, replace=False)
        image_orient_subset = images[indices]

        for k, image in enumerate(image_orient_subset):
            convolved = apply_filter(image, fil)
            curr_transformed.append(convolved.flatten())

        curr_transformed = np.hstack(curr_transformed)
        # intermediate subsample to limit size
        curr_transformed = np.sort(curr_transformed)[np.round(np.linspace(0, curr_transformed.size - 1, min(curr_transformed.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        transformed = np.concatenate((transformed, curr_transformed))

    # append the negation of all coefficients to account for other orientations
    neg_transformed = -1 * transformed
    transformed = np.concatenate((transformed, neg_transformed))
    
    # select max of CONSTANT_SAMPLE_SIZE coefs
    transformed = np.sort(transformed)[np.round(np.linspace(0, transformed.size - 1, min(transformed.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
    filter_data_map[i] = transformed

filter_data_map[0]

  0%|          | 0/42 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Saving Data

In [ ]:
data_output_file = os.path.join(ROOT_DIR, 'transformed-data', f"{FINAL_DATA_NAME}-gabor.pickle") 
size_output_file = os.path.join(ROOT_DIR, 'transformed-data', f"{FINAL_DATA_NAME}-gabor-size.pickle")

total_samples = np.prod(images.shape)
print("Without subsampling size:", total_samples)
print("With subsampling, per-filter:", len(filter_data_map[0]))
pd.to_pickle(filter_data_map, data_output_file)
pd.to_pickle({i : total_samples for i in range(len(filter_data_map))}, size_output_file)

Without subsampling size: 786432000
With subsampling, per-filter: 100000
